# Predictions ships

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
from vessel_manoeuvring_models.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
import matplotlib
#plt.style.use('presentation')

from vessel_manoeuvring_models.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig


from myst_nb import glue
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from vessel_manoeuvring_models.models.regression import MotionRegression, Regression

from vessel_manoeuvring_models.parameters import df_parameters
from vessel_manoeuvring_models.substitute_dynamic_symbols import run
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
p = df_parameters["symbol"]
import statsmodels.api as sm

from vessel_manoeuvring_models.models.force_from_motion import predict_force

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)
from vessel_manoeuvring_models.prime_system import PrimeSystem
from wPCC_pipeline.pipelines.kvlcc2.nodes import calculate_thrust

from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import initial_state, extended_kalman_filter, extended_kalman_smoother, guess_covariance_matrixes

model_tests_all = runs_globals['model_test_ids']
model_tests_all.pop('generic_kvlcc2')
model_tests_all.pop('tanker')
model_tests_all.pop('kvlcc2')

In [ ]:


ships = model_tests_all.keys()

ships

In [ ]:
%reload_kedro

def load_model_tests_ship(ship = 'wpcc'):
    
    model_tests = model_tests_all[ship]
    
    dfs = []
    
    for id in model_tests:
        
        df_ = catalog.load(f"{ship}.{id}.data")

            
        df_['id'] = id
        dfs.append(df_)
            
    df = pd.concat(dfs)
    
    return df


In [ ]:
def select_model_tests(df, n=9):
    df['|psi|'] = df['psi'].abs()
    
    indexes = df.groupby(by='id')['|psi|'].max().sort_values(ascending=False).index[0:n]
    mask = df['id'].isin(indexes)
    df_select = df[mask]
    return df_select

In [ ]:
def plot_ship(df_select, ship, vmm="vmm_abkowitz"):
    
    fig,axes = plt.subplots(ncols=3, nrows=3)
    fig.set_size_inches(14,10)
    axes = axes.flatten()
    for ax, (id, df_) in zip(axes,df_select.groupby(by='id')):
        
        prediction = catalog.load(f"{ship}.updated.{vmm}.joined.{id}.data_resimulate")

        
        dataframes = {
            'model test':df_,
            'prediction':prediction,
            
        }
        
        track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'], ax=ax, N=4)
        ax.get_legend().set_visible(False)
        ax.set_title(None)
    
    axes[0].legend()
    fig.suptitle(f'Track plots: {ship}')
    plt.tight_layout()
    

In [ ]:
for ship, model_tests in model_tests_all.items():
    
    df = load_model_tests_ship(ship=ship)
    ship_data = catalog.load(f"{ship}.ship_data")
    df_select = select_model_tests(df=df)
    plot_ship(df_select, ship, vmm='vmm_abkowitz')
    
    